In [11]:
import pandas as pd
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn.functional as F
import numpy as np
import re

In [13]:
# Загрузка данных
df = pd.read_csv('datamart.csv', delimiter=';')
# df = pd.read_excel('Перечень фичей .xlsx')
df = df.dropna().reset_index(drop=True)


# Подготовка данных
# df['text'] = df['name'] + ': ' + df['description']
X = df['description'].tolist()
y = df['name'].tolist()
df

,name,description,schema_name,table_name,created_timestamp
0,balance_features,Среднегодовые балансы,l_profiling,prod_flake_balance,2023-08-07 12:17:23.798
1,atm_cash_management,Рассчёт фичей для модели внесения и выдачи нал...,l_profiling,fs_atm_cash_management,2023-10-31 16:05:27.536
2,rates_prop,Информация по относительной разнице ставок офф...,l_profiling,fs__rates_prop,2023-10-18 15:52:50.672
3,owner_features,Признаки по владельцам компаний,l_profiling,prod_flake_owner_features,2023-10-03 11:01:50.374
4,out_datamart,Профайлинги для клиентов ММБ,l_profiling,prod_flake_mmb_features,2022-12-13 20:17:30.388
...,...,...,...,...,...
235,anket_ul_features,Витрина фичей внешнего анкетного модуля ЮЛ для...,l_aaa_risk,anket_ul_ul_model_features,2024-07-05 17:20:41.192
236,goszakupka_scores,Витрина по скорам госзакупок,l_profiling,prod_flake_goszakupka_score,2024-07-08 17:53:51.660
237,fs_client_profile_daily_features,"Ежедневные фичи по клиентам: по адресу, катего...",l_profiling,fs_client_profile_daily_features,2024-07-08 18:48:54.344
238,triggers_table,Признаки на основе данных НСПК МИР,l_profiling,prod_flake_nspk_features,2024-07-09 17:55:34.619


In [15]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Разделение на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [17]:
# Загрузка токенизатора и модели
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=len(le.classes_))
model.config.pad_token_id = tokenizer.eos_token_id  # Устанавливаем pad_token_id



Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
# Токенизация данных
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=512)


In [21]:
# Создание датасетов
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, y_train)
test_dataset = Dataset(test_encodings, y_test)


In [13]:
# training_args = TrainingArguments(
#     output_dir='./results',  # Директория для сохранения результатов обучения
#     num_train_epochs=2,  # Количество эпох обучения
#     per_device_train_batch_size=16,  # Размер батча для обучения на одном устройстве
#     per_device_eval_batch_size=16,  # Размер батча для оценки на одном устройстве
#     warmup_steps=1000,  # Количество шагов для разогрева оптимизатора
#     weight_decay=0.03,  # Коэффициент L2-регуляризации для предотвращения переобучения
#     logging_dir='./logs',  # Директория для сохранения логов
#     logging_steps=10,  # Частота шагов для логирования
#     learning_rate=5e-5,  # Скорость обучения (learning rate)
#     gradient_accumulation_steps=4,  # Количество шагов для накопления градиентов перед обновлением весов
#     eval_strategy="steps",  # Стратегия оценки: проводить оценку каждые n шагов
#     eval_steps=500,  # Частота шагов для проведения оценки
#     save_strategy="steps",  # Стратегия сохранения: сохранять модель каждые n шагов
#     save_steps=500,  # Частота шагов для сохранения модели
#     save_total_limit=2,  # Максимальное количество сохраняемых чекпоинтов
#     load_best_model_at_end=True,  # Загружать лучшую модель в конце обучения
#     metric_for_best_model="eval_loss",  # Метрика для определения лучшей модели
#     greater_is_better=False,  # False означает, что меньшее значение метрики лучше (например, для loss)
# )

In [15]:
# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs=2,
#     per_device_train_batch_size=12,  # Увеличен размер батча, если позволяет память GPU
#     per_device_eval_batch_size=12,  # Увеличен размер батча для оценки
#     warmup_steps=500,  # Еще уменьшено количество шагов разогрева
#     logging_dir='./logs',
#     logging_steps=200,  # Увеличен интервал логирования
#     learning_rate=0.01,  # Еще увеличена скорость обучения
#     gradient_accumulation_steps=1,  # Убрано накопление градиентов для ускорения
#     eval_strategy="steps",
#     eval_steps=1500,  # Увеличен интервал оценки
#     save_strategy="steps",
#     save_steps=1500,  # Увеличен интервал сохранения
#     save_total_limit=1,
#     load_best_model_at_end=True,
#     metric_for_best_model="eval_loss",
#     greater_is_better=False,
#     # fp16=True,  # Включено смешанное обучение с половинной точностью
#     dataloader_num_workers=4,  # Добавлены рабочие процессы для загрузки данных
#     disable_tqdm=False,
#     optim="adamw_torch",  # Использование оптимизированной версии AdamW
#     # bf16=True,  # Использование bfloat16 для еще большего ускорения на поддерживаемых GPU
#     gradient_checkpointing=True,  # Включение градиентных чекпоинтов для экономии памяти
#     max_grad_norm=1.0,  # Ограничение нормы градиента для стабильности
# )

In [17]:

# Настройка параметров обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    learning_rate=0.01,
    weight_decay=0.05,
    logging_dir='./logs',
    logging_steps=10,
    # fp16=True,  # Включено смешанное обучение с половинной точностью
    # dataloader_num_workers=4,  # Увеличено количество рабочих процессов для загрузки данных
    disable_tqdm=False
)

# Инициализация Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# # Обучение модели
# trainer.train()


In [19]:
# Обучение модели
trainer.train()

Step,Training Loss
25,6.983500


TrainOutput(global_step=48, training_loss=6.137722810109456, metrics={'train_runtime': 264.6754, 'train_samples_per_second': 1.451, 'train_steps_per_second': 0.181, 'total_flos': 40832222625792.0, 'train_loss': 6.137722810109456, 'epoch': 2.0})

In [23]:
# Сохранение модели
model_path = "./fine_tuned_gpt2_model_datamart_test240" 
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('./fine_tuned_gpt2_model_datamart_test240\\tokenizer_config.json',
 './fine_tuned_gpt2_model_datamart_test240\\special_tokens_map.json',
 './fine_tuned_gpt2_model_datamart_test240\\vocab.json',
 './fine_tuned_gpt2_model_datamart_test240\\merges.txt',
 './fine_tuned_gpt2_model_datamart_test240\\added_tokens.json')

In [38]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# # Загрузка модели и токенизатора
# model_path = "./fine_tuned_gpt2_model_2"  # путь к вашей сохраненной модели
# model = GPT2LMHeadModel.from_pretrained(model_path)
# tokenizer = GPT2Tokenizer.from_pretrained(model_path)

model_path = "./fine_tuned_gpt2_model_datamart_test240"

# Загрузка токенизатора и модели
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2ForSequenceClassification.from_pretrained(model_path, num_labels=len(le.classes_))
model.config.pad_token_id = tokenizer.eos_token_id  # Устанавливаем pad_token_id



# Убедимся, что у токенизатора есть pad_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [40]:

torch.manual_seed(1)
np.random.seed(1)

# Функция для предсказания
def predict(text, top_k=5):
    # Токенизация входного текста
    inputs = tokenizer(text, truncation=True, padding=True, max_length=512, return_tensors="pt")
    
    # Получение выходных данных модели
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Применение softmax для получения вероятностей
    probabilities = F.softmax(outputs.logits, dim=1)
    
    # Получение top_k наиболее вероятных классов и их вероятностей
    top_probs, top_indices = torch.topk(probabilities, k=top_k)
    
    # Преобразование индексов обратно в метки классов
    top_classes = le.inverse_transform(top_indices[0].numpy())
    
    # Создание списка кортежей (класс, вероятность)
    results = list(zip(top_classes, top_probs[0].numpy()))
    
    # Сортировка результатов по убыванию вероятности
    results.sort(key=lambda x: x[1], reverse=True)
    
    return results

# Пример использования
text = "какие таблицы использовать для расчета CLTV по клиенту"
top_predictions = predict(text, top_k=5)

print("Топ подходящих вариантов:")
for i, (class_name, probability) in enumerate(top_predictions, 1):
    print(f"{i}. {class_name}: {probability:.4f}")
    ind = df[df['name']==class_name]['description'].index[0]
    
    print(df[df['name']==class_name]['description'][ind])
    
# у тебя в базе знаний есть информация о данных, посоветуй мне какие данные лучше выбрать для расчета модели оттока?
# # Пример использования
# print(predict("какие таблицы использовать для расчета CLTV по клиенту"))

Топ подходящих вариантов:
1. cardtransactions_v2: 0.3517
Признаки по карточным транзакциям
2. cardtransactions_basic: 0.2013
Признаки по карточным транзакциям
3. app_vars: 0.1562
Скрипты wsrm_features от Никиты Трифонова
4. statement: 0.0517
Признаки по счетам клиентов
5. out_datamart_30: 0.0290
Дополнительные транзакционные признаки


### Упаковка в телегу

In [43]:
import os
import telebot
from telebot import types

In [45]:
# bot = telebot.TeleBot("6191958363:AAHb8GhePS5MDqCoh3zg-BkhDdFc11M5j4A")
bot = telebot.TeleBot("6127210948:AAEs1JKu0Lz8eFzEsHcO_TcSb9vMHKdtNIA")

In [ ]:

@bot.message_handler(func=lambda message: True)
def handle_message(message):
    text = message.text
    top_predictions = predict(text, top_k=5)
    
    response = "Топ подходящих вариантов:\n\n"
    for i, (class_name, probability) in enumerate(top_predictions, 1):
        response += f"{i}. {class_name}: {probability:.4f}\n"
        ind = df[df['name']==class_name]['description'].index[0]
        response += f"{df[df['name']==class_name]['description'][ind]}\n\n"
    
    bot.reply_to(message, response)

bot.polling()